# L'effet d'un outlier sur un modèle de prédiction

*high leverage point*

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
from sklearn import linear_model

plt.rcParams["figure.figsize"] = [3, 3]

nous regardons l'impact du taux de criminalité des quartiers sur les prix des logements

les données contiennent les colonnes suivantes

```python
['HousePrice', 'HsPrc($10,000)', 'CrimeRate', 'MilesPhila', 'PopChg', 'Name', 'County']
```

nous ne sommes intéressés que par

```python
['Taux de criminalité', 'Prix de la maison']
```

notre question :
Quelle est l'influence de la colonne `CrimeRate` sur la colonne `HousePrice` ?

on lit le `csv` en ne gardant que les colonnes `CrimeRate` and `HousePrice`  
utilisation du paramètre `usecols` de `pandas.read_csv`

In [ ]:
filename = 'Philadelphia_Crime_Rate_noNA.csv'
philadelphia = pd.read_csv(filename,
                           usecols=['CrimeRate', 'HousePrice'],
                           sep=',')

In [ ]:
philadelphia.head()

In [ ]:
philadelphia.shape

In [ ]:
philadelphia.dtypes

In [ ]:
philadelphia.describe()

In [ ]:
philadelphia[['HousePrice']].boxplot(figsize=(2, 2))
plt.show()

In [ ]:
# plt.boxplot?

In [ ]:
philadelphia[['CrimeRate']].boxplot(figsize=(2, 2))
plt.show()

In [ ]:
philadelphia[['CrimeRate', 'HousePrice',]].hist(bins=10)
plt.tight_layout()
plt.show()

In [ ]:
philadelphia.plot.scatter(x='CrimeRate', y='HousePrice', marker='.')
plt.show()

## prediction en présence de l'outlier

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
philadelphia.shape

In [ ]:
philadelphia[['CrimeRate']].shape

In [ ]:
X = philadelphia[['CrimeRate']] # we input a dataframe
y = philadelphia['HousePrice']  # we predict a colonne

In [ ]:
X_std = StandardScaler().fit_transform(X)

In [ ]:
X_std = StandardScaler().fit(X).transform(X)

il faudrait utiliser un pipeline  
on le laisse en exercice

In [ ]:
regr = LinearRegression()

In [ ]:
regr = regr.fit(X_std, y)

on regarde les valeurs des paramètres `intercept_` et `coef_` qui ont été entraînés

In [ ]:
regr.intercept_, regr.coef_[0]

on prédit nos valeurs pour `y`

In [ ]:
y_predict = regr.predict(X_std)

In [ ]:
from sklearn.metrics import mean_squared_error

on calcule l'erreur

In [ ]:
mean_squared_error(y_predict, y) # ouh elle est très grande

on calcule le score  
on choisit *the coefficient of determination $r^2$ of the prediction*  
soit la méthode `score` de la regression  
soit la fonction `r2_score` de `metrics`

In [ ]:
from sklearn.metrics import r2_score

ce score est défini comme $1 - \dfrac{u}{v}$ où
- $u$ est la MSE entre $y$ et $y_{pred}$
- $v$ est ((y_true - y_true.mean()) ** 2).sum()

un modèle constant qui prédirait toujours l'espérance de `y`, quelle que soit l'entrée a un score de $0$  
le meilleur score est à $1$

dans l'implémentation `score` ou `metrics.r2_score` si $v$ est $0$ (`y` constant) la fonction renvoie, pr défaut,  $1$ 

In [ ]:
u = np.sum(np.square(y - y_predict))
v = np.sum(np.square(y - np.mean(y)))

1 - (u/v)

In [ ]:
regr.score(X_std, y)

In [ ]:
r2_score(y, y_predict)

il est très mauvais il rejoint le score naïf

on plot le `y` prédit et le `y` mesuré (celui du csv)  
en fonction de la colonne `CrimeRate`

In [ ]:
plt.plot(X_std, y, 'b.')
plt.plot(X_std, y_predict, 'r-'); # la droite des prédictions
plt.show()

idem si on plot la ligne `y = intercept + coeff_0 X`

In [ ]:
plt.plot(X_std, y, 'b.')
plt.plot([np.min(X_std), np.max(X_std)],
         [(regr.intercept_ + regr.coef_[0] * i) for i in [np.min(X_std), np.max(X_std)]], 'g-');
plt.show()

la droite de regression est clairement attirée par le point extrême 

## prediction en absence de l'outlier

on va enlever notre outlier

In [ ]:
philadelphia['CrimeRate'].argmax() # on le cherche

In [ ]:
philadelphia['CrimeRate'].max()

In [ ]:
philadelphia.iloc[62] # c'est donc lui

In [ ]:
philadelphia.drop(62, axis=0, inplace=True) # on le drop (axis à 0 parce qu'on drop une observation)
                                            # (1 ce sont les colones)

et on recommence tout ...

In [ ]:
philadelphia.plot.scatter(x='CrimeRate', y='HousePrice', marker='.', figsize=(3, 3))
plt.show()

In [ ]:
X = philadelphia[['CrimeRate']] # we input a dataframe
y = philadelphia['HousePrice']  # we predict a colonne

X_std = StandardScaler().fit_transform(X)
regr = linear_model.LinearRegression()
regr = regr.fit(X_std, y)

y_predict = regr.predict(X_std)

regr.score(X_std, y)

il est un peu moins mauvais

In [ ]:
plt.plot(X_std, y, 'b.')
plt.plot(X_std, y_predict, 'r-'); # la droite des prédictions

idem si on plot la ligne `y = intercept + coeff_0 X`

In [ ]:
plt.plot(X_std, y, 'b.')
plt.plot([np.min(X_std), np.max(X_std)],
         [(regr.intercept_ + regr.coef_[0] * i) for i in [np.min(X_std), np.max(X_std)]], 'g-');
plt.show()

END